# **Single-molecule fluorescence simulator**

<font size = 4>My man, if you want to simulate single molecules, this is da bomb!

# **1. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive. 

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Run this cell to connect your Google Drive to Colab

#@markdown * Click on the URL. 

#@markdown * Sign in your Google Account. 

#@markdown * Copy the authorization code. 

#@markdown * Enter the authorization code. 

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive". 

#mounts user's Google Drive to Google Colab.

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **2. Install Deep-STORM and dependencies**
---


In [ ]:
#@markdown ##Install S&M dependencies

# Import common libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import scipy.io as sio
# from os.path import abspath
# from sklearn.model_selection import train_test_split
from skimage import io
import time
import os
# import shutil
# import csv
# from PIL import Image
# from PIL.TiffTags import TAGS
# from scipy.ndimage import gaussian_filter
import math
# from astropy.visualization import simple_norm
# from sys import getsizeof

# # For sliders and dropdown menu, progress bar
from ipywidgets import interact
import ipywidgets as widgets
from tqdm import tqdm

# # For Multi-threading in simulation
from numba import njit, prange




# Multi-threaded Erf-based image construction
@njit(parallel=True)
def FromLoc2Image_MultiThreaded(xc_array, yc_array, photon_array, sigma_array, image_size = 64, pixel_size = 100):
  Image = np.zeros((image_size, image_size))
  for ij in prange(image_size*image_size):
    j = int(ij/image_size)
    i = ij - j*image_size
    for (xc, yc, photon, sigma) in zip(xc_array, yc_array, photon_array, sigma_array):
      # Don't bother if the emitter has photons <= 0 or if Sigma <= 0
      if (photon > 0) and (sigma > 0):
        S = sigma*math.sqrt(2)
        x = i*pixel_size - xc
        y = j*pixel_size - yc
        # Don't bother if the emitter is further than 4 sigma from the centre of the pixel
        if (x+pixel_size/2)**2 + (y+pixel_size/2)**2 < 16*sigma**2:
          ErfX = math.erf((x+pixel_size)/S) - math.erf(x/S)
          ErfY = math.erf((y+pixel_size)/S) - math.erf(y/S)
          Image[j][i] += 0.25*photon*ErfX*ErfY
  return Image



def getPixelSizeTIFFmetadata(TIFFpath, display=False):
  with Image.open(TIFFpath) as img:
    meta_dict = {TAGS[key] : img.tag[key] for key in img.tag.keys()}

  # TIFF tags
  # https://www.loc.gov/preservation/digital/formats/content/tiff_tags.shtml
  # https://www.awaresystems.be/imaging/tiff/tifftags/resolutionunit.html
  ResolutionUnit = meta_dict['ResolutionUnit'][0] # unit of resolution
  xResolution = meta_dict['XResolution'][0][0] # number of pixels / ResolutionUnit
  width = meta_dict['ImageWidth'][0]
  height = meta_dict['ImageLength'][0]

  if ResolutionUnit == 2:
    # Units given are in inches
    pixel_size = 0.025*1e9/xResolution
  elif ResolutionUnit == 3:
    # Units given are in cm
    pixel_size = 0.01*1e9/xResolution
  else: 
    # ResolutionUnit is therefore 1
    # Units are not determined and it is assumed it is in metres
    pixel_size = 1e9/xResolution

  if display:
    print('Pixel size obtained from metadata: '+str(pixel_size)+' nm')
  
  return (pixel_size, width, height)




print('--------------------------------')
print('S&M installation complete.')


--------------------------------
S&M installation complete.



# **3. Generate S&M simulations**
---




## **3.1 Simulate data**
---
This simulation tool allows you to generate SMLM data of randomly distrubuted emitters in a field-of-view. 
The assumptions are as follows:

*   Gaussian Point Spread Function (PSF) with `Sigma = 0.21 x Lambda / NA`
*   Each emitter will emit `n_photons` per frame, and generate their equivalent Poisson noise.
*   The camera will contribute Gaussian noise to the signal with a standard deviation defined by `ReadOutNoise_ADC` in ADC
*   The `emitter_density` is defined as the number of emitters / um^2 on any given frame
*   The `n_photons` and `wavelength` can additionally include some Gaussian variability by setting `wavelength_std` and `n_photons_std`

Important note:
- All dimensions are in nanometer (e.g. `FOV_size` = 6400 represents a field of view of 6.4 um x 6.4 um)
- `emitter_density` **is defined in number of emitters / um^2**



In [ ]:

# ---------------------------- User input ----------------------------
#@markdown Run the simulation
#@markdown --- 
#@markdown Camera settings: 
FOV_size = 6400 #@param {type:"number"}
pixel_size = 100 #@param {type:"number"}
ADC_per_photon_conversion = 1.0 #@param {type:"number"}
ReadOutNoise_ADC = 50 #@param {type:"number"}
ADC_offset =  100#@param {type:"number"}

#@markdown Acquisition settings: 
emitter_density =  6#@param {type:"number"}
number_of_frames =  500#@param {type:"integer"}
NA =  1.4 #@param {type:"number"}
wavelength =  700#@param {type:"number"}
wavelength_std =  50#@param {type:"number"}
n_photons =  5000#@param {type:"number"}
n_photons_std =  50#@param {type:"number"}
#@markdown Dynamics settings: 
velocity = 0.01#@param {type:"number"}
bounce_off_walls = True #@param {type:"boolean"}



# ---------------------------- Variable initialisation ----------------------------
# Start the clock to measure how long it takes
start = time.time()

print('-----------------------------------------------------------')
n_molecules = round(emitter_density*FOV_size*FOV_size/10**6)
print('Number of molecules / FOV: '+str(n_molecules))

sigma = 0.21*wavelength/NA
sigma_std = 0.21*wavelength_std/NA
print('Gaussian PSF sigma: '+str(round(sigma,2))+' +/- '+str(round(sigma_std,2))+' nm')

M = N = round(FOV_size/pixel_size)
print('Final image size: '+str(M)+'x'+str(M))


np.random.seed(1)
display_upsampling = 8 # used to display the loc map here
NoiseFreeImages = np.zeros((number_of_frames, M, M))
locImage = np.zeros((number_of_frames, display_upsampling*M, display_upsampling*N))
# locImage_noUpsampling = np.zeros((number_of_frames, M, N))

frames = []
all_xloc = []
all_yloc = []
all_photons = []
all_sigmas = []



# ---------------------------- Main simulation loop ----------------------------
print('-----------------------------------------------------------')
for f in tqdm(range(number_of_frames)):
  
  if (f == 0):
    # Define the coordinates of emitters at t=0 by randomly distributing them across the FOV
    x_c = np.random.uniform(low=0.0, high=FOV_size, size=n_molecules)
    y_c = np.random.uniform(low=0.0, high=FOV_size, size=n_molecules)

    # Define each molecules direction of movement
    theta = np.random.uniform(low=0.0, high=2*math.pi, size=n_molecules)

    photon_array = np.random.normal(n_photons, n_photons_std, size=n_molecules)
    sigma_array = np.random.normal(sigma, sigma_std, size=n_molecules)
    # x_c = np.linspace(0,3000,5)
    # y_c = np.linspace(0,3000,5)
  else:
    x_c = x_c + velocity*np.cos(theta)
    y_c = y_c + velocity*np.sin(theta)

    if bounce_off_walls:
      theta[x_c < 0] = math.pi-theta[x_c < 0]
      theta[x_c > FOV_size] = math.pi-theta[x_c > FOV_size]

      theta[y_c < 0] = -theta[y_c < 0]
      theta[y_c > FOV_size] = -theta[y_c > FOV_size]


  all_xloc += x_c.tolist()
  all_yloc += y_c.tolist()
  frames += ((f+1)*np.ones(x_c.shape[0])).tolist()
  all_photons += photon_array.tolist()
  all_sigmas += sigma_array.tolist()

  # Get the approximated locations according to the grid pixel size
  Chr_emitters = [int(max(min(round(display_upsampling*x_c[i]/pixel_size),N*display_upsampling-1),0)) for i in range(len(x_c))]
  Rhr_emitters = [int(max(min(round(display_upsampling*y_c[i]/pixel_size),M*display_upsampling-1),0)) for i in range(len(y_c))]

  # # Get the approximated locations according to the grid pixel size
  # Chr_emitters_noUpsampling = [int(max(min(round(display_upsampling*x_c[i]/pixel_size),N*display_upsampling-1),0)) for i in range(len(x_c))]
  # Rhr_emitters_noUpsampling = [int(max(min(round(display_upsampling*y_c[i]/pixel_size),M*display_upsampling-1),0)) for i in range(len(y_c))]


  # Build Localization image
  for (r,c) in zip(Rhr_emitters, Chr_emitters):
    locImage[f][r][c] += 1

  # NoiseFreeImages[f] = FromLoc2Image_MultiThreaded(x_c, y_c, M, pixel_size, sigma)
  # heatmaps[f] = FromLoc2Image_MultiThreaded(x_c, y_c, upsampling_factor*M, pixel_size/upsampling_factor, sigma_heatmap)
  NoiseFreeImages[f] = FromLoc2Image_MultiThreaded(x_c, y_c, photon_array, sigma_array, M, pixel_size)
  # heatmaps[f] = FromLoc2Image_MultiThreaded2(x_c, y_c, photon_array, sigma_heatmap*np.ones(n_molecules), upsampling_factor*image_size, pixel_size/upsampling_factor)


# ---------------------------- Create DataFrame for localization file ----------------------------
# Table with localization info as dataframe output
LocData = pd.DataFrame()
LocData["frame"] = frames
LocData["x [nm]"] = all_xloc
LocData["y [nm]"] = all_yloc
LocData["Photon #"] = all_photons
LocData["Sigma [nm]"] = all_sigmas
LocData.index += 1  # set indices to start at 1 and not 0 (same as ThunderSTORM)

# ---------------------------- Create DataFrame for simulation parameters ----------------------------

# ---------------------------- Estimation of SNR ----------------------------
n_frames_for_SNR = 100
M_SNR = 10
x_c = np.random.uniform(low=0.0, high=pixel_size*M_SNR, size=n_frames_for_SNR)
y_c = np.random.uniform(low=0.0, high=pixel_size*M_SNR, size=n_frames_for_SNR)
SNR = np.zeros(n_frames_for_SNR)
for i in range(n_frames_for_SNR):
  SingleEmitterImage = FromLoc2Image_MultiThreaded([x_c[i]], [x_c[i]], [n_photons], [sigma], M_SNR, pixel_size)
  Signal_photon = np.max(SingleEmitterImage)
  Noise_photon = math.sqrt((ReadOutNoise_ADC/ADC_per_photon_conversion)**2 + Signal_photon)
  SNR[i] = Signal_photon/Noise_photon


print('SNR: '+str(np.mean(SNR))+' +/- '+str(np.std(SNR)))
# ---------------------------- ----------------------------


# Table with info
simParameters = pd.DataFrame()
simParameters["FOV size (nm)"] = [FOV_size]
simParameters["Pixel size (nm)"] = [pixel_size]
simParameters["ADC/photon"] = [ADC_per_photon_conversion]
simParameters["Read-out noise (ADC)"] = [ReadOutNoise_ADC]
simParameters["Constant offset (ADC)"] = [ADC_offset]

simParameters["Emitter density (emitters/um^2)"] = [emitter_density]
simParameters["Number of frames"] = [number_of_frames]
simParameters["NA"] = [NA]
simParameters["Wavelength (nm)"] = [wavelength]
simParameters["STD of wavelength (nm)"] = [wavelength_std]
simParameters["Number of photons"] = [n_photons]
simParameters["STD of number of photons"] = [n_photons_std]
simParameters["Velocity (nm/frame)"] = [velocity]
simParameters["Bounce off?"] = [bounce_off_walls]

simParameters["Sigma (nm))"] = [sigma]
simParameters["STD of Sigma (nm))"] = [sigma_std]
simParameters["SNR"] = [np.mean(SNR)]
simParameters["STD of SNR"] = [np.std(SNR)]



# ---------------------------- Finish simulation ----------------------------
# Calculating the noisy image
Images = ADC_per_photon_conversion * np.random.poisson(NoiseFreeImages) + ReadOutNoise_ADC * np.random.normal(size = (number_of_frames, M, N)) + ADC_offset

# Convert to 16-bit
Images[Images > 65535] = 65535
Images[Images <= 0] = 0
Images = Images.astype(np.uint16)


# ---------------------------- Display ----------------------------
# Displaying the time elapsed for simulation
dt = time.time() - start
minutes, seconds = divmod(dt, 60) 
hours, minutes = divmod(minutes, 60) 
print("Time elapsed:",hours, "hour(s)",minutes,"min(s)",round(seconds,1),"sec(s)")


# Interactively display the results using Widgets
def scroll_in_time(frame):
  f = plt.figure(figsize=(18,6))
  plt.subplot(1,4,1)
  plt.imshow(locImage[frame-1], interpolation='bilinear', vmin = 0, vmax=0.1)
  plt.title('Localization image')
  plt.axis('off');

  plt.subplot(1,4,2)
  plt.imshow(NoiseFreeImages[frame-1], interpolation='nearest')
  plt.title('Noise-free simulation')
  plt.axis('off');

  plt.subplot(1,4,3)
  plt.imshow(Images[frame-1], interpolation='nearest')
  plt.title('Noisy simulation')
  plt.axis('off');

interact(scroll_in_time, frame=widgets.IntSlider(min=1, max=Images.shape[0], step=1, value=0, continuous_update=False));

# Display the head of the dataframe with localizations
LocData.tail()


-----------------------------------------------------------
Number of molecules / FOV: 246
Gaussian PSF sigma: 105.0 +/- 7.5 nm
Final image size: 64x64


  0%|          | 0/500 [00:00<?, ?it/s]

-----------------------------------------------------------


100%|██████████| 500/500 [00:03<00:00, 130.08it/s]
/usr/local/lib/python3.6/dist-packages/numba/ir_utils.py:2041: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'photon_array' of function 'FromLoc2Image_MultiThreaded'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-63-ae5322ccd42b>", line 36:
@njit(parallel=True)
def FromLoc2Image_MultiThreaded(xc_array, yc_array, photon_array, sigma_array, image_size = 64, pixel_size = 100):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/usr/local/lib/python3.6/dist-packages/numba/ir_utils.py:2041: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'sigma_array' of function 'FromLoc2Image_MultiThreaded'.

For more information visit http:/

SNR: 11.210068716529323 +/- 0.6231133669201893
Time elapsed: 0.0 hour(s) 0.0 min(s) 6.4 sec(s)


interactive(children=(IntSlider(value=1, continuous_update=False, description='frame', max=500, min=1), Output…

,frame,x [nm],y [nm],Photon #,Sigma [nm]
122996,500.0,5714.436343,64.820634,4988.788211,103.943169
122997,500.0,3696.765848,3686.485227,4983.696935,100.035811
122998,500.0,1172.834102,1991.994085,4998.454429,106.942739
122999,500.0,5047.481467,3308.935977,5017.785863,105.108363
123000,500.0,3913.456396,5861.483733,5042.479342,93.903150


In [ ]:
# @markdown ---
# @markdown #Play this cell to save the simulated stack
# @markdown ####Please select a path to the folder where to save the simulated data.
Save_path = "/content/gdrive/My Drive/Simulations/2020-06-05 Linearly diffusing particles SNR 5" #@param {type:"string"}
Data_name = "Sim1_d6_v50_bounce-SNR5" #@param {type:"string"}

io.imsave(os.path.join(Save_path, Data_name+'.tif'),Images)
LocData.to_csv(os.path.join(Save_path, Data_name+'.csv'))
simParameters.to_csv(os.path.join(Save_path, Data_name+'_sim_parameters.csv'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: /content/gdrive/My Drive/Simulations/2020-06-05 Linearly diffusing particles SNR 5/Sim1_d6_v50_bounce-SNR5.tif is a low contrast image
  import sys


## **6.2. Download your predictions**
---

<font size = 4>**Store your data** and ALL its results elsewhere by downloading it from Google Drive and after that clean the original folder tree (datasets, results, trained model etc.) if you plan to train or use new networks. Please note that the notebook will otherwise **OVERWRITE** all files which have the same name.


#**Thank you for using S&M!**